In [31]:
# Gemini API
!pip install -q google-genai

In [32]:
!pip install -U google-generativeai

In [33]:
# DB 연결 및 데이터 처리
# sqlalchemy가 DB 접속을 관리하고, psycopg2-binary는 PostgreSQL 드라이버
!pip install -q pandas sqlalchemy psycopg2-binary

print("✅ 라이브러리 설치 완료.")

✅ 라이브러리 설치 완료.


In [34]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from google.colab import userdata  # Colab Secrets
from google.colab import auth
from google import genai
import json
import time

In [35]:
# Google Cloud 인증
# Cloud SQL Auth Proxy 사용을 위해 Colab 사용자를 인증합니다.
auth.authenticate_user()
print("✅ Colab 사용자 인증 완료.")

✅ Colab 사용자 인증 완료.


In [36]:
# Colab Secrets에서 정보 불러오기
try:
    # Cloud SQL (PostgreSQL) 접속 정보
    DB_USER = userdata.get('POSTGRES_USER')        # 사용자 계정 이름
    DB_PASS = userdata.get('POSTGRES_PASS')        # 사용자 계정 처음 만들 때 설정한 비밀번호
    DB_NAME = userdata.get('POSTGRES_DB_NAME')     # DB 이름

    # GCP 인스턴스 연결 이름
    # (GCP 콘솔 > Cloud SQL > 인스턴스 개요 페이지에서 확인)
    INSTANCE_CONNECTION_NAME = userdata.get('INSTANCE_CONNECTION_NAME')

    # Gemini API 키
    os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

    if not all([DB_USER, DB_PASS, DB_NAME, INSTANCE_CONNECTION_NAME, os.environ["GEMINI_API_KEY"]]):
        raise ValueError("Colab Secrets에 모든 값이 설정되었는지 확인하세요.")

except Exception as e:
    print(f"❌ Colab Secrets 로드 실패: {e}")
    raise

In [37]:
# LLM 모델 설정
client = genai.Client()
MODEL_NAME = "gemini-2.5-flash"

# DB 연결 (Auth Proxy 사용)
DB_HOST = "127.0.0.1"  # Auth Proxy는 항상 127.0.0.1 (localhost)을 사용
DB_PORT = "5432"       # PostgreSQL 기본 포트
DB_ENGINE = "postgresql+psycopg2" # psycopg2 드라이버 사용

# [요청하신 DB_URL 형식 (psycopg3 사용)]
# DB_URL 형식은 psycopg2와 psycopg3가 거의 동일합니다.
# (참고: SQLAlchemy는 psycopg2-binary를 더 안정적으로 지원합니다)
DATABASE_URL = f"{DB_ENGINE}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

print(f"✅ 설정 완료. DB URL 형식: {DB_ENGINE}://...@{DB_HOST}:{DB_PORT}/{DB_NAME}")
print(f"✅ LLM 모델: {MODEL_NAME}")

✅ 설정 완료. DB URL 형식: postgresql+psycopg2://...@127.0.0.1:5432/insightbee
✅ LLM 모델: gemini-2.5-flash


In [38]:
TEST_PROMPT_KR = """
### 역할
너는 취업준비생들의 시간 절약을 도와주기 위해 기업의 기사를 감정분석해주는 시스템이야. 취업준비생들이 기업 분석에 필요한 원본 정보와 기사의 감정분석을 동시에 제공받아 취업에 도움되도록 돕는 것이 너의 역할이야.

-----------------------------
### 분류 규칙
아래의 "기사 제목", "기사 본문", "작성일"을 종합해 다음 필드를 **취업준비생의 입장에서** 정확히 채워.

- company: 기사의 핵심 대상 기업명 (정식 기업명으로 정규화)
- title : 기사 제목 (원본 그대로 출력)
- sentiment: 기사의 전반적인 감정. 다음 중 하나: **`긍정 (Positive)`**, **`부정 (Negative)`**, **`중립 (Neutral)`**
- sentiment_score: 'sentiment'의 점수 (float 타입, -1.0에서 1.0 사이)
- sentiment_reason: 'sentiment'의 근거. 감정을 그렇게 판단한 핵심적인 근거 문장과 이유를 명확히 제시 (최대 100자).
- body : 기사 본문 내용

-----------------------------
### 기업명 정규화 목록 (허용 값)
※ 기사에서 발견되는 약어 및 계열사를 아래 정식 명칭으로 통일해

- 삼성전자:삼전, 삼성전자(주), 삼성전자 반도체, 005930
- SK하이닉스:하이닉스, SK하이닉스(주), 000660
- LG에너지솔루션:LG엔솔, 엔솔, 373220
- 삼성바이오로직스:삼바, 바오로직스, 207940
- 삼성전자우:삼전우, 삼성전자 우선주, 005935
- 두산에너빌리티:두산빌리티, 두에너, 두산중공업, 034020
- 현대차:현대자동차, 005380
- 한화에어로스페이스:한화에어로, 012450
- HD현대중공업:현대중공업, 329180
- KB금융:KB, 국민은행 지주, 105560
- 기아:기아자동차, 000270
- NAVER:네이버, 035420
- 한화오션:대우조선해양 (舊), 042660
- 셀트리온:셀트, 068270
- 신한지주:신한은행 지주, 055550
- SK스퀘어:SK C&C, 402340
- HD현대일렉트릭:현대일렉트릭, 267260
- HD한국조선해양:한국조선해양, 009540
- 삼성생명:삼성생, 032830
- 한국전력:한전, 015760
- 카카오:카카오(주), 035720
- LG화학:LG화학(주), 051910
- 삼성SDI:SDI, 006400
- 현대모비스:모비스, 012330
- POSCO홀딩스:포스코, 005490
- 삼성중공업:삼성重, 010140
- 현대로템:로템, 064350
- 하나금융지주:하나금융, 086790
- SK이노베이션:SK이노, 096770
- 효성중공업:효중, 298040
- 삼성화재:삼성화, 000810
- 메리츠금융지주:메리츠금융, 138040
- 고려아연:영풍제련소, 010130
- 포스코퓨처엠:포퓨, 포스코케미칼 (舊), 003670
- 우리금융지주:우리금융, 316140
- HMM:현대상선 (舊), 011200
- SK:SK(주), SK 지주사, 034730
- 삼성전기:삼전기, 009150
- HD현대:HD현대 지주, 267270
- 두산:두산 지주, 000150
- KT&G:033780
- 기업은행:IBK, 024110
- 미래에셋증권:미래에셋, 006800
- 삼성에스디에스:삼성SDS, 018260
- LG전자:LG전, 066570
- LS ELECTRIC:LS산전 (舊), 010120
- 하이브:하이브 엔터, 빅히트 (舊), 352820
- 한미반:한미반, 042700
- 크래프톤:크래프톤(주), 259960

※ 위 목록에 없는 기업명은 기사에 언급된 정식/일반 명칭을 그대로 출력.
-----------------------------
### 정규화/추론 규칙 (중요)
- company 정리:
  - 기사에 언급된 약어나 계열사 명칭이 '기업명 정규화 목록'에 있으면 반드시 허용된 정식 명칭으로 통일
  - 목록에 없는 기업은 기사에 언급된 명칭을 그대로 사용
- sentiment:
  - 기사 내용 전체를 고려하되, 취업준비생에게 미치는 장기적/단기적 영향을 중심으로 분류.(예: 당장의 호재여도 장기적 리스크가 보이면 '중립' 이하로 평가)
 - 기사의 제목과 내용을 고려해서 감정분석.
- sentiment_score:
 - 'sentiment'의 점수
 - float 타입 (-1.0 ~ +1.0), '긍정'이면 양수, '부정'이면 음수, '중립'이면 0.0에 가까운 값
- sentiment_reason:
 - 'sentiment'의 핵심적인 근거 문장과 이유를 제시.
- 형식:
  - 모든 문자열은 큰따옴표로 감싸고, 내부 큰따옴표는 이스케이프(`\"`)
  - 트레일링 콤마 금지, 키 순서는 자유지만 모든 키 포함
  - 마크다운/설명/코드펜스 출력 금지 (JSON만 출력)

-----------------------------
### Few-shot 예시(긍정)

#### input
{{
  "title": "삼성전자 3분기 영업익 12.2조…HBM3E 엔비디아 납품 공식화(종합)",
  "body": "삼성전자[005930]는 연결 기준 올해 3분기 영업이익이 12조1천661억원으로 지난해 동기보다 32.5% 증가한 것으로 집계됐다고 30일 공시했다. 이번 영업이익은 연합인포맥스가 집계한 시장 전망치 10조4천832억원을 16.1% 상회했다. 매출은 86조617억원으로 작년 동기 대비 8.8% 증가하며 역대 최대 분기 매출을 기록했다. 순이익은 12조2천257억원으로 21% 늘었다. 사업부별로 반도체 사업을 담당하는 디바이스솔루션(DS) 부문은 매출 33조1천억원, 영업이익 7조원을 기록했다. 메모리는 HBM3E 판매 확대와 DDR5, 서버용 솔리드 스테이트 드라이브(SSD) 등의 수요 강세로 사상 최고 분기 매출을 기록했다. 영업이익은 제품 가격 상승과 전 분기 발생했던 재고 관련 일회성 비용이 감소하면서 큰 폭으로 개선됐다. 특히 HBM3E는 전 고객 대상으로 양산 판매 중이라고 삼성전자는 설명했다. 삼성전자는 그간 5세대 제품인 HBM3E 12단을 '큰손' 엔비디아에 납품하기 위해 공을 들여왔다. 삼성전자는 또 6세대인 HBM4에 대해 샘플을 요청한 모든 고객사에 샘플을 출하했다고 밝혔다. 시스템LSI는 주요 고객사의 프리미엄 라인업에 시스템 온 칩(SoC)을 안정적으로 공급했으나, 시장 전반의 재고 조정과 계절적 수요 둔화로 실적은 정체됐다. 파운드리는 첨단공정 중심으로 분기 최대 수주 실적을 달성했으며, 일회성 비용이 감소하고 라인 가동률이 개선되면서 전 분기 대비 실적이 개선됐다. 디바이스 경험(DX) 부문은 폴더블 신모델 출시 효과와 견조한 플래그십 스마트폰 판매 등으로 매출 48조4천억원, 영업이익 3조5천억원을 기록했다. 모바일경험(MX)은 Z폴드7 판매 호조로 전 분기 및 전년 동기 대비 매출과 영업이익이 모두 성장했다. 영상디스플레이(VD)는 Neo QLED, OLED, 대형 TV 등 프리미엄 제품 판매가 견조했으나, TV 시장 수요 정체와 경쟁 심화로 전 분기 대비 실적이 감소했다. 생활가전은 계절적 비수기 진입과 미국 관세 영향으로 전 분기 대비 영업이익이 감소했다. 하만은 소비자 오디오 제품 판매 호조와 전장 부문의 매출 확대로 매출 4조원, 영업이익 4천억원을 기록했다. 디스플레이는 중소형의 경우 플래그십 스마트폰의 견조한 수요와 신제품 출시 대응으로 판매가 확대되며 매출 8조1천억원, 영업이익 1조2천억원을 올렸다. 삼성전자는 4분기에 인공지능(AI) 산업의 급속한 성장으로 인해 DS, DX 부문 모두 새로운 시장 기회가 열릴 것으로 예상한다고 밝혔다. 메모리의 경우 D램은 AI 및 서버 수요에 적극 대응할 계획으로 HBM3E와 고용량 서버 DDR5 제품 중심으로 판매를 확대할 계획이다. 시스템LSI는 프리미엄용 SoC와 이미지센서 판매 확대를 추진한다. 파운드리는 2나노 양산을 본격화하고 가동률 향상 및 원가 개선을 통해 실적 개선을 이어갈 예정이다. 이에 더해 2026년에는 AI 투자 확대로 반도체 경기 호조가 지속될 것으로 예상했다. 특히 HBM4 수요 또한 증가할 것으로 전망돼 1c 캐파 확대를 통해 적극 대응할 예정이다. 삼성전자는 HBM 판매를 지속 확대하고 차별화된 성능 기반의 HBM4 양산에 집중할 방침이다. 또한, AI용 DDR5, LPDDR5x, GDDR7 등 고부가 가치 제품 판매 비중을 확대할 계획이다. 파운드리는 2나노 신제품과 HBM4 베이스 다이 양산에 집중하며 미국 테일러 팹을 2026년부터 본격적으로 가동할 예정이다. 삼성전자는 미래 성장을 위해 연구개발 투자를 지속하며 3분기 누계 기준 역대 최대인 26조9천억원의 연구개발비를 집행했다. 환율의 경우 전 분기 대비 원화 강세로 달러 거래 비중이 높은 DS 부문에서 소폭 부정적 영향이 있었으나, DX 부문에서 일부 긍정적 영향이 발생해 전사 전체 영업이익에 미치는 영향은 미미했다고 삼성전자는 설명했다.",
}}

#### output
{{
  "company": "삼성전자",
  "sentiment": "긍정 (Positive)",
  "sentiment_score":0.9,
  "sentiment_reason": "시장 예상치를 상회하는 12.2조 원의 영업이익과 역대 최대 매출을 기록했고, 핵심인 HBM3E의 엔비디아 납품 공식화로 미래 성장성이 확실함.",
  "body": "삼성전자[005930]는 연결 기준 올해 3분기 영업이익이 12조1천661억원으로 지난해 동기보다 32.5% 증가한 것으로 집계됐다고 30일 공시했다. 이번 영업이익은 연합인포맥스가 집계한 시장 전망치 10조4천832억원을 16.1% 상회했다. 매출은 86조617억원으로 작년 동기 대비 8.8% 증가하며 역대 최대 분기 매출을 기록했다. 순이익은 12조2천257억원으로 21% 늘었다. 사업부별로 반도체 사업을 담당하는 디바이스솔루션(DS) 부문은 매출 33조1천억원, 영업이익 7조원을 기록했다. 메모리는 HBM3E 판매 확대와 DDR5, 서버용 솔리드 스테이트 드라이브(SSD) 등의 수요 강세로 사상 최고 분기 매출을 기록했다. 영업이익은 제품 가격 상승과 전 분기 발생했던 재고 관련 일회성 비용이 감소하면서 큰 폭으로 개선됐다. 특히 HBM3E는 전 고객 대상으로 양산 판매 중이라고 삼성전자는 설명했다. 삼성전자는 그간 5세대 제품인 HBM3E 12단을 '큰손' 엔비디아에 납품하기 위해 공을 들여왔다. 삼성전자는 또 6세대인 HBM4에 대해 샘플을 요청한 모든 고객사에 샘플을 출하했다고 밝혔다. 시스템LSI는 주요 고객사의 프리미엄 라인업에 시스템 온 칩(SoC)을 안정적으로 공급했으나, 시장 전반의 재고 조정과 계절적 수요 둔화로 실적은 정체됐다. 파운드리는 첨단공정 중심으로 분기 최대 수주 실적을 달성했으며, 일회성 비용이 감소하고 라인 가동률이 개선되면서 전 분기 대비 실적이 개선됐다. 디바이스 경험(DX) 부문은 폴더블 신모델 출시 효과와 견조한 플래그십 스마트폰 판매 등으로 매출 48조4천억원, 영업이익 3조5천억원을 기록했다. 모바일경험(MX)은 Z폴드7 판매 호조로 전 분기 및 전년 동기 대비 매출과 영업이익이 모두 성장했다. 영상디스플레이(VD)는 Neo QLED, OLED, 대형 TV 등 프리미엄 제품 판매가 견조했으나, TV 시장 수요 정체와 경쟁 심화로 전 분기 대비 실적이 감소했다. 생활가전은 계절적 비수기 진입과 미국 관세 영향으로 전 분기 대비 영업이익이 감소했다. 하만은 소비자 오디오 제품 판매 호조와 전장 부문의 매출 확대로 매출 4조원, 영업이익 4천억원을 기록했다. 디스플레이는 중소형의 경우 플래그십 스마트폰의 견조한 수요와 신제품 출시 대응으로 판매가 확대되며 매출 8조1천억원, 영업이익 1조2천억원을 올렸다. 삼성전자는 4분기에 인공지능(AI) 산업의 급속한 성장으로 인해 DS, DX 부문 모두 새로운 시장 기회가 열릴 것으로 예상한다고 밝혔다. 메모리의 경우 D램은 AI 및 서버 수요에 적극 대응할 계획으로 HBM3E와 고용량 서버 DDR5 제품 중심으로 판매를 확대할 계획이다. 시스템LSI는 프리미엄용 SoC와 이미지센서 판매 확대를 추진한다. 파운드리는 2나노 양산을 본격화하고 가동률 향상 및 원가 개선을 통해 실적 개선을 이어갈 예정이다. 이에 더해 2026년에는 AI 투자 확대로 반도체 경기 호조가 지속될 것으로 예상했다. 특히 HBM4 수요 또한 증가할 것으로 전망돼 1c 캐파 확대를 통해 적극 대응할 예정이다. 삼성전자는 HBM 판매를 지속 확대하고 차별화된 성능 기반의 HBM4 양산에 집중할 방침이다. 또한, AI용 DDR5, LPDDR5x, GDDR7 등 고부가 가치 제품 판매 비중을 확대할 계획이다. 파운드리는 2나노 신제품과 HBM4 베이스 다이 양산에 집중하며 미국 테일러 팹을 2026년부터 본격적으로 가동할 예정이다. 삼성전자는 미래 성장을 위해 연구개발 투자를 지속하며 3분기 누계 기준 역대 최대인 26조9천억원의 연구개발비를 집행했다. 환율의 경우 전 분기 대비 원화 강세로 달러 거래 비중이 높은 DS 부문에서 소폭 부정적 영향이 있었으나, DX 부문에서 일부 긍정적 영향이 발생해 전사 전체 영업이익에 미치는 영향은 미미했다고 삼성전자는 설명했다."
}}

### Few-shot 예시(부정)

#### input
{{
  "title": "'하이브 상장’으로 떼돈 번 방시혁 측근...수사 시작되자 잠적 [지금이뉴스]",
  "body": "방시혁 하이브 의장과 상장을 통한 이익 배분을 주도하고 계획한 것으로 알려진 인물인 김중동 전 하이브 최고투자책임자(CIO)가 방 의장에 대한 수사가 본격화된 올해 6월 이후 잠적했습니다.
 한국경제의 보도에 따르면, 미국 시민권자인 김 전 CIO는 해외에 거주하며 경찰과 금융당국의 조사 요청에도 응하지 않고 있는 것으로 알려졌습니다.
방 의장과 자본시장 사이를 넘나들며 프로젝트를 총괄한 김 전 CIO는 그 대가로 1000억원에 달하는 막대한 수익을 거둔 '키맨'으로 꼽힙니다.
서울경찰청 금융범죄수사대는 방 의장의 사기적 부정거래 혐의 사건과 관련한 핵심 인물인 김 전 CIO에 대한 조사만 진행하지 못한 것으로 알려졌습니다.
김 전 CIO는 지난 6월 미국으로 출국한 것으로 전해졌는데, 이 시기는 경찰이 방 의장에 대해 압수수색 영장을 신청했다는 사실이 알려진 시점으로 수사가 임박하자 이를 피하기 위해 출국했을 가능성도 제기되고 있습니다.
현재 김 전 CIO는 방 의장에 대한 수사가 본격화된 이후엔 사실상 업계에서 자취를 감춘 후 미국 로스엔젤레스(LA) 지역에 머무는 것으로 알려졌습니다.
앞서 김 전 CIO는 SV인베스트먼트에서 하이브 초기 투자를 담당했던 인물로 방 의장과 투자자들 사이에서 사모펀드(PEF) 설립과 하이브의 상장, 방 의장과의 이익 공유까지 모든 프로젝트를 설계한 키맨으로 알려졌습니다.
방 의장은 개인재산 운용도 김 전 CIO 조언을 따를 만큼 신임한 것으로 알려졌습니다.
 현재 수사 당국은 김 전 CIO의 직접 조사를 재차 추진하고 있지만 법조계에선 미국 시민권자인 김 전 CIO를 송환 받기 어려울 것으로 전망하고 있습니다.",
}}

#### output
{{
  "company": "하이브",
  "sentiment": "부정(Negative)",
  "sentiment_score":-0.85,
  "sentiment_reason": "방시혁 의장 관련 사기적 부정거래 혐의 수사가 본격화된 시점에 핵심 인물인 전 CIO가 잠적하여 경영 투명성 및 지배구조 리스크 심화.",
  "body": "방시혁 하이브 의장과 상장을 통한 이익 배분을 주도하고 계획한 것으로 알려진 인물인 김중동 전 하이브 최고투자책임자(CIO)가 방 의장에 대한 수사가 본격화된 올해 6월 이후 잠적했습니다.
 한국경제의 보도에 따르면, 미국 시민권자인 김 전 CIO는 해외에 거주하며 경찰과 금융당국의 조사 요청에도 응하지 않고 있는 것으로 알려졌습니다.
방 의장과 자본시장 사이를 넘나들며 프로젝트를 총괄한 김 전 CIO는 그 대가로 1000억원에 달하는 막대한 수익을 거둔 '키맨'으로 꼽힙니다.
서울경찰청 금융범죄수사대는 방 의장의 사기적 부정거래 혐의 사건과 관련한 핵심 인물인 김 전 CIO에 대한 조사만 진행하지 못한 것으로 알려졌습니다.
김 전 CIO는 지난 6월 미국으로 출국한 것으로 전해졌는데, 이 시기는 경찰이 방 의장에 대해 압수수색 영장을 신청했다는 사실이 알려진 시점으로 수사가 임박하자 이를 피하기 위해 출국했을 가능성도 제기되고 있습니다.
현재 김 전 CIO는 방 의장에 대한 수사가 본격화된 이후엔 사실상 업계에서 자취를 감춘 후 미국 로스엔젤레스(LA) 지역에 머무는 것으로 알려졌습니다.
앞서 김 전 CIO는 SV인베스트먼트에서 하이브 초기 투자를 담당했던 인물로 방 의장과 투자자들 사이에서 사모펀드(PEF) 설립과 하이브의 상장, 방 의장과의 이익 공유까지 모든 프로젝트를 설계한 키맨으로 알려졌습니다.
방 의장은 개인재산 운용도 김 전 CIO 조언을 따를 만큼 신임한 것으로 알려졌습니다.
 현재 수사 당국은 김 전 CIO의 직접 조사를 재차 추진하고 있지만 법조계에선 미국 시민권자인 김 전 CIO를 송환 받기 어려울 것으로 전망하고 있습니다."
}}


### Few-shot 예시(중립)

#### input
{{
  "title": "한전, 2025 APEC 정상회의 무정전 전력공급",
  "body": "한국전력이 2025 APEC(아시아태평양경제협력체) 정상회의 기간 동안 무결점 전력 운영을 달성하며 정부·지방자치단체·유관기관과의 공조를 통해 국가적 대형 국제행사의 성공적 운영을 든든히 뒷받침했다.
한전은 지난 1일 종료된 '2025 APEC 정상회의'와 전국 주요 도시에서 열린 고위·장관급 실무자 회의(총 24회) 기간 동안 단 한건의 정전 없이 안정적인 전력공급을 완수했다고 3일 밝혔다.
한전은 지난해 6월 APEC 개최지가 경주로 확정된 이후 비상 전력대응체제를 가동했다. 올해 2월 경주에서 열린 제1차 고위관리회의를 시작으로 부산·제주·인천·서울 등에서 진행된 모든 회의마다 전력확보 상황실을 운영하고 공급선로 계통 점검과 설비 진단, 현장 비상대기 등을 시행하여 무정전 운영에 만전을 기했다.이 과정에서 연인원 6400여명의 인력을 투입, 안정적인 행사 운영을 지원했다.
올해 7월부터는 APEC 전담조직을 구성하고 'Safe Power! Successful APEC!' 이라는 슬로건 아래 전사 비상대응 체계를 운영했다.
지능형 배전망 관리시스템(ADMS)을 활용해 주요 행사장과 정상 숙소에 대한 전력공급 모니터링을 시행하고 GPS 기반 출동관리 시스템과 특별기동대를 가동하여 상황 발생 시 즉각 대응이 가능한 체계를 갖췄다.
특히 정상회의가 열린 경주 보문관광단지의 전력 안정성 강화를 위해 약 100억원을 투입해 신규 배전선로 설치, 노후설비 교체, 과부하 해소와 미관 개선 공사를 지난 9월 완료했다.
또한 경상북도·경주시·한국전기안전공사 등 유관기관의 긴밀한 협력체계를 구축하고, 경상북도 APEC 준비지원단에 전문 기술인력을 파견해 전력 점검과 대응체계를 운영했다.
행사장 전력설비 검측, 전담인력 상시배치, 비상발전기와 UPS 포함 4중 전력체계 구축, 24시간 상황실 운영 등을 통해 비상대응 능력을 강화했다.
김동철 한전사장은 "2025 APEC 정상회의와 전국 고위·장관급 회의에서 무정전을 달성할 수 있었던 것은 현장 기술진과 전 직원의 헌신 덕분"이라며 "앞으로도 국가 중요행사에 안정적으로 전력을 공급하여 대한민국의 국제적 신뢰도를 높여 나가겠다"고 밝혔다.",
}}

#### output
{{
  "company": "한국전력",
  "sentiment": "중립(Neutral)",
  "sentiment_score":0.2,
  "sentiment_reason": "공기업으로서의 의무를 충실히 이행했다는 점에서 약한 긍정으로 볼 수 있으나, 재무적 영향이 없어 중립으로 분류합니다.",
  "body": "한국전력이 2025 APEC(아시아태평양경제협력체) 정상회의 기간 동안 무결점 전력 운영을 달성하며 정부·지방자치단체·유관기관과의 공조를 통해 국가적 대형 국제행사의 성공적 운영을 든든히 뒷받침했다.
한전은 지난 1일 종료된 '2025 APEC 정상회의'와 전국 주요 도시에서 열린 고위·장관급 실무자 회의(총 24회) 기간 동안 단 한건의 정전 없이 안정적인 전력공급을 완수했다고 3일 밝혔다.
한전은 지난해 6월 APEC 개최지가 경주로 확정된 이후 비상 전력대응체제를 가동했다. 올해 2월 경주에서 열린 제1차 고위관리회의를 시작으로 부산·제주·인천·서울 등에서 진행된 모든 회의마다 전력확보 상황실을 운영하고 공급선로 계통 점검과 설비 진단, 현장 비상대기 등을 시행하여 무정전 운영에 만전을 기했다.이 과정에서 연인원 6400여명의 인력을 투입, 안정적인 행사 운영을 지원했다.
올해 7월부터는 APEC 전담조직을 구성하고 'Safe Power! Successful APEC!' 이라는 슬로건 아래 전사 비상대응 체계를 운영했다.
지능형 배전망 관리시스템(ADMS)을 활용해 주요 행사장과 정상 숙소에 대한 전력공급 모니터링을 시행하고 GPS 기반 출동관리 시스템과 특별기동대를 가동하여 상황 발생 시 즉각 대응이 가능한 체계를 갖췄다.
특히 정상회의가 열린 경주 보문관광단지의 전력 안정성 강화를 위해 약 100억원을 투입해 신규 배전선로 설치, 노후설비 교체, 과부하 해소와 미관 개선 공사를 지난 9월 완료했다.
또한 경상북도·경주시·한국전기안전공사 등 유관기관의 긴밀한 협력체계를 구축하고, 경상북도 APEC 준비지원단에 전문 기술인력을 파견해 전력 점검과 대응체계를 운영했다.
행사장 전력설비 검측, 전담인력 상시배치, 비상발전기와 UPS 포함 4중 전력체계 구축, 24시간 상황실 운영 등을 통해 비상대응 능력을 강화했다.
김동철 한전사장은 "2025 APEC 정상회의와 전국 고위·장관급 회의에서 무정전을 달성할 수 있었던 것은 현장 기술진과 전 직원의 헌신 덕분"이라며 "앞으로도 국가 중요행사에 안정적으로 전력을 공급하여 대한민국의 국제적 신뢰도를 높여 나가겠다"고 밝혔다."
}}


-----------------------------
### Input data
- 기사 제목: {title}
- 기사 본문: {body}

-----------------------------
### Output 형식(반드시 JSON만 출력)
{{
  "company": "",
  "title":"",
  "sentiment": "",
  "sentiment_score":"",
  "sentiment_reason": "",
  "body": ""
}}
"""
print("✅ LLM 프롬프트")

✅ LLM 프롬프트


In [39]:
RESPONSE_SCHEMA = {
    "type": "object",
    "properties": {
        "company": {"type": "string"},
        "title": {"type": "string"},
        "sentiment": {"type": "string"},
        "sentiment_reason": {"type": "string"},
        "sentiment_score": {"type": "number"},
        "body": {"type": "string"},
    },
    "required": ["company", "title", "sentiment", "sentiment_reason", "sentiment_score", "body"]
}

print("✅ RESPONSE_SCHEMA 변수 정의 완료.")

✅ RESPONSE_SCHEMA 변수 정의 완료.


In [40]:
# Auth Proxy 바이너리 다운로드 및 권한 설정
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
print("Auth Proxy 다운로드 완료.")

--2025-11-14 17:06:57--  https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64
Resolving dl.google.com (dl.google.com)... 173.194.193.91, 173.194.193.136, 173.194.193.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.193.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24830551 (24M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  23.68M  --.-KB/s    in 0.1s    

2025-11-14 17:06:58 (227 MB/s) - ‘cloud_sql_proxy’ saved [24830551/24830551]

Auth Proxy 다운로드 완료.


In [41]:
# Auth Proxy 백그라운드 실행
# (Colab 세션이 이 Proxy를 통해 127.0.0.1:5432로 Cloud SQL에 접속합니다)
get_ipython().system_raw(
    f'./cloud_sql_proxy -instances={INSTANCE_CONNECTION_NAME}=tcp:{DB_PORT} &'
)
print(f"Auth Proxy 실행 중... (인스턴스: {INSTANCE_CONNECTION_NAME})")
time.sleep(5)

Auth Proxy 실행 중... (인스턴스: sesac-yoojikol:us-central1:insightbee)


In [42]:
# SQLAlchemy 엔진 생성
try:
    engine = create_engine(DATABASE_URL)

    # 간단한 쿼리로 연결 테스트
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print(f"✅ DB 연결 테스트 성공 (결과: {result.scalar()})")

except Exception as e:
    print(f"❌ DB 연결 실패: {e}")
    print("👉 DB 접속 정보(사용자명, 비밀번호, DB명, 인스턴스 연결 이름)를 다시 확인하세요.")
    raise

✅ DB 연결 테스트 성공 (결과: 1)


In [43]:
# SQL - DB 전체 테이블 목록 확인용

import pandas as pd
from sqlalchemy import text

print("--- [ 현재 데이터베이스의 'public' 스키마에 있는 모든 테이블 목록 ] ---")

# PostgreSQL에서 'public' 스키마의 모든 테이블 목록을 조회하는 쿼리
LIST_TABLES_QUERY = text("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
""")

try:
    df_tables = pd.read_sql(LIST_TABLES_QUERY, engine)

    if len(df_tables) == 0:
        print("👉 'public' 스키마에 테이블이 없습니다. DB가 비어있거나 다른 스키마에 있을 수 있습니다.")
    else:
        print("✅ 테이블 목록 조회를 성공했습니다.")
        display(df_tables) # 테이블 목록을 DataFrame으로 깔끔하게 보여줌

except NameError:
    print("❌ 'engine' 변수가 정의되지 않았습니다.")
except Exception as e:
    print(f"❌ 테이블 목록 조회 실패: {e}")
    print("👉 DB 접속 정보(특히 DB_NAME)가 정확한지 다시 확인해 보세요.")

--- [ 현재 데이터베이스의 'public' 스키마에 있는 모든 테이블 목록 ] ---
✅ 테이블 목록 조회를 성공했습니다.


,table_name
0,article_analysis
1,article_topics
2,article_topics_link
3,articles
4,companies
5,news_articles
6,topics


In [44]:
# SQL - Colab에서 DB 직접 확인용 스크립트

print("--- [ companies 테이블 상위 10개 확인 ] ---")
try:
    df_comp = pd.read_sql(text("SELECT * FROM companies LIMIT 10"), engine)
    display(df_comp)
except Exception as e:
    print(f"❌ companies 테이블 확인 실패: {e}")

# print("\n--- [ news_articles 테이블 상위 10개 확인 ] ---")
# try:
#     df_art = pd.read_sql(text("SELECT * FROM news_articles LIMIT 10"), engine)
#     display(df_art)
# except Exception as e:
#     print(f"❌ article_analysis 테이블 확인 실패: {e}")

print("\n--- [ article_analysis 테이블 상위 10개 확인 ] ---")
try:
    df_analy = pd.read_sql(text("SELECT * FROM article_analysis LIMIT 10"), engine)
    display(df_analy)
except Exception as e:
    print(f"❌ article_analysis 테이블 확인 실패: {e}")

--- [ companies 테이블 상위 10개 확인 ] ---


,id,name_ko,name_en,corp_code,stock_code,created_at
0,5,케이티,KT Cooatio,00186461,,2025-10-27 15:18:11.599053
1,6,카카오,Kakao.Corp,00918444,,2025-10-27 15:18:11.599053
2,7,우리금융지주,"WooriFinance Holdings Co.,Ltd.",00375302,053000,2025-10-27 15:18:11.599053
3,8,삼성물산,Samsung C&T Corporation,00126229,000830,2025-10-27 15:18:11.599053
4,9,SK,"SK Holdings Co., Ltd.",00144155,003600,2025-10-27 15:18:11.599053
5,10,기아,KIA,01664948,,2025-10-27 15:18:11.599053
6,11,메리츠금융지주,MERITZ FINANCIAL GROUP INC.,00860332,138040,2025-10-27 15:18:11.599053
7,12,삼성전기,"SAMSUNG ELECTRO-MECHANICS CO.,LTD",00126371,009150,2025-10-27 15:18:11.599053
8,13,현대로템,Hyundai-Rotem Co.,00302926,064350,2025-10-27 15:18:11.599053
9,14,알테오젠,Alteogen Inc.,00989619,196170,2025-10-27 15:18:11.599053



--- [ article_analysis 테이블 상위 10개 확인 ] ---


,article_id,sentiment,sentiment_reason,sentiment_status,summary,summary_status,topic_status,last_processed_at
0,1,긍정 (Positive),"한·미 관세 협상 극적 타결로 자동차 관세율 인하, 대미 투자 약정 불확실성 해소,...",COMPLETED,None,PENDING,PENDING,None
1,2,긍정 (Positive),한미 관세 협상 타결로 자동차 관세가 25%에서 15%로 인하되어 현대차그룹의 연간...,COMPLETED,None,PENDING,PENDING,None
2,3,긍정 (Positive),"한미 관세협상 타결로 국내 수출 기업, 특히 자동차 업계의 고관세 부담이 해소되어 ...",COMPLETED,None,PENDING,PENDING,None
3,4,긍정 (Positive),대미 자동차 수출 관세가 25%에서 15%로 인하되면서 현대차그룹의 관세 비용 부담...,COMPLETED,None,PENDING,PENDING,None
4,5,긍정 (Positive),한미 관세 협상 타결로 미국 대 한국 자동차 품목 관세가 15%로 인하되어 현대차와...,COMPLETED,None,PENDING,PENDING,None
5,6,긍정 (Positive),한미 양국이 자동차 관세를 25%에서 15%로 낮추기로 합의하여 현대차그룹의 연간 ...,COMPLETED,None,PENDING,PENDING,None
6,7,긍정 (Positive),한미 관세 협상 타결로 미국 자동차 품목 관세가 15% 인하되면서 현대차와 기아의 ...,COMPLETED,None,PENDING,PENDING,None
7,8,긍정 (Positive),해외 IB가 코스피 6000선 도달을 전망하며 한국 증시의 상승 여력이 크다고 분석...,COMPLETED,None,PENDING,PENDING,None
8,9,긍정 (Positive),한미 관세 협상 타결로 현대차·기아 등 자동차 업계는 연간 3조원 이상 비용 절감 ...,COMPLETED,None,PENDING,PENDING,None
9,10,긍정 (Positive),한-미 관세 협상 타결로 자동차·부품 관세가 인하되어 주가가 14% 상승하는 등 긍...,COMPLETED,None,PENDING,PENDING,None


In [45]:
# DB에서 기사 데이터 로드
print("\n--- 'news_articles' 테이블에서 데이터 로드 중... ---")
try:
    query = text("SELECT 	article_id, title, content FROM news_articles")

    # [테스트용] 데이터가 너무 많으면, 우선 5~10개만 가져와서 테스트하세요.
    # query = text("SELECT id, title, content, date FROM news_articles LIMIT 5")

    df = pd.read_sql(query, engine)

    if df.empty:
        print("⚠️ 'news_articles' 테이블에 데이터가 없습니다. 분석을 중단합니다.")
        raise SystemExit(0) # 데이터 없으면 중지

    print(f"✅ {len(df)}개의 기사 로드 완료.")
    print(df.head())

except Exception as e:
    print(f"❌ 데이터 로드 실패: {e}")
    print("👉 'news_articles' 테이블과 'id', 'title', 'content', 'date' 컬럼이 존재하는지 확인하세요.")
    raise


--- 'news_articles' 테이블에서 데이터 로드 중... ---
✅ 2694개의 기사 로드 완료.
   article_id                                              title  \
0           1                [사설] 한·미 관세 협상 극적 타결…수출·외환 큰 부담 덜었다   
1           2  한미 관세 타결, 車관세 15%…현대차그룹 영업익 연 2.4조 증가 전망 < 자동차...   
2           3  자동차업계 ‘천만다행’… 되살아난 수출경기 탄력 기대감 [2025 경주 에이펙-29...   
3           4     車 관세 15%로 인하, 업계 안도…현대차그룹 "정부에 감사" [APEC 2025]   
4           5                                    현대기아차 애프태마켓서 급등   

                                             content  
0  [사설] 한·미 관세 협상 극적 타결…수출·외환 큰 부담 덜었다\n-\n기사 스크랩...  
1  한미 정상회담을 계기로 지연되던 관세 협상이 타결됐다. 이번 타결로 이르면 11월부...  
2  가격경쟁력 확보… 4분기 개선 가능성\n현대차그룹 “정부, 헌신적 노력 감사”\n반...  
3  車 관세 15%로 인하, 업계 안도…현대차그룹 "정부에 감사" [APEC 2025]...  
4  현대기아차 애프태마켓서 급등\n-\n기사 스크랩\n-\n공유\n-\n댓글\n-\n클...  


In [46]:
# 네트워크 프록시 충돌 해결
print("\n--- 네트워크 프록시 충돌 해결 ---")
print("Colab의 'auth.authenticate_user()'와 'Gemini API' 간의 프록시 충돌을 해결합니다.")

import os

# Gemini API가 로컬 프록시(localhost)를 통하지 않고
# 인터넷으로 직접 통신하도록 설정합니다.
# (generativeai.googleapis.com은 프록시를 사용하지 말라고 명시)

# 1. 기존 NO_PROXY 변수가 있다면, 거기에 Google API 호스트를 추가합니다.
existing_no_proxy = os.environ.get('NO_PROXY', '')
google_api_host = 'generativeai.googleapis.com'

if google_api_host not in existing_no_proxy:
    # 쉼표로 구분하여 추가
    new_no_proxy = f"{existing_no_proxy},{google_api_host}" if existing_no_proxy else google_api_host
    os.environ['NO_PROXY'] = new_no_proxy
    print(f"✅ NO_PROXY 환경 변수 설정 완료: {os.environ['NO_PROXY']}")

# 만약의 경우를 대비해, Colab이 설정할 수 있는 HTTP(S)_PROXY 해제.
# (Gemini API는 API 키를 사용하므로 이 변수들이 필요 없습니다)
removed_http = os.environ.pop('HTTP_PROXY', None)
removed_https = os.environ.pop('HTTPS_PROXY', None)

if removed_http or removed_https:
    print(f"✅ 기존 HTTP_PROXY/HTTPS_PROXY 환경 변수 해제 완료.")
else:
    print(f"✅ (HTTP_PROXY/HTTPS_PROXY 변수는 설정되지 않았음)")

print("✅ 네트워크 환경 설정 완료. 이제 LLM 모델을 로드하세요.")


--- 네트워크 프록시 충돌 해결 ---
Colab의 'auth.authenticate_user()'와 'Gemini API' 간의 프록시 충돌을 해결합니다.
✅ (HTTP_PROXY/HTTPS_PROXY 변수는 설정되지 않았음)
✅ 네트워크 환경 설정 완료. 이제 LLM 모델을 로드하세요.


In [47]:
# LLM 모델 재설정
print("\n--- LLM 모델 재설정 ---")
print("참고: 'gemini-2.5-flash' 모델은 'genai.GenerativeModel'을 사용해야 합니다.")
print("       'genai.Client()'는 이전 PaLM API용입니다.")

# 'google.genai'를 명시적으로 import (이전 셀에서 'from google import genai'를 사용했다면)
# from google import genai # This import is not needed if we use google.generativeai
import google.generativeai as genai

# API 키는 이미 'os.environ["GEMINI_API_KEY"]'에 설정되어 있습니다.
# genai.configure()는 환경 변수에 키가 있으면 자동으로 로드합니다.
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
except Exception as e:
    print("❌ Gemini API 키 설정 실패. Colab Secrets를 다시 확인하세요.")
    raise e

# JSON 모드 설정을 포함하여 모델 다시 정의
generation_config_json = genai.GenerationConfig(
    response_mime_type="application/json",
    response_schema=RESPONSE_SCHEMA # 이전에 정의한 스키마 사용
)

# 올바른 모델 객체 생성
model_json = genai.GenerativeModel(
    MODEL_NAME, # 이전에 정의한 "gemini-2.5-flash"
    generation_config=generation_config_json
)

print(f"✅ LLM '{MODEL_NAME}' (JSON 모드) 재설정 완료.")


--- LLM 모델 재설정 ---
참고: 'gemini-2.5-flash' 모델은 'genai.GenerativeModel'을 사용해야 합니다.
       'genai.Client()'는 이전 PaLM API용입니다.
✅ LLM 'gemini-2.5-flash' (JSON 모드) 재설정 완료.


In [48]:
# 네트워크 오류 처리를 위한 모듈 임포트
from http.client import RemoteDisconnected
# ConnectionResetError는 Python 내장 오류라 별도 임포트가 필요 없을 수 있습니다.
print("✅ 네트워크 재시도에 필요한 모듈 준비 완료.")

✅ 네트워크 재시도에 필요한 모듈 준비 완료.


In [49]:
# # 네트워크 오류 자동 재시도 함수
# print("\n--- 자동 재시도 함수 정의 ---")
# print("   (네트워크 오류 발생 시 3회 자동 재시도)")

# def analyze_article_json_final(article_id, article_title, article_content, model):
#     """
#     Gemini 모델을 호출하여 기사를 분석합니다.
#     - 네트워크 오류(ConnectionResetError, RemoteDisconnected) 발생 시
#       '지수 백오프'를 적용하여 최대 3회 자동 재시도합니다.
#     """

#     # 입력 값 검증
#     if not all([article_title, article_content]):
#         print(f"  (ID: {article_id}) ⚠️ 입력 값(제목 또는 본문)이 비어있어 건너뜁니다.")
#         return None

#     # 프롬프트 생성
#     try:
#         final_prompt = TEST_PROMPT_KR.replace("{title}", article_title) \
#                                    .replace("{body}", article_content)
#     except Exception as e:
#         print(f"  (ID: {article_id}) ❌ 프롬프트 생성 오류: {e}")
#         return None

#     # 자동 재시도 루프 설정
#     MAX_RETRIES = 3
#     BASE_DELAY_SECONDS = 3 # (기본 3초 대기)

#     for attempt in range(MAX_RETRIES):
#         try:
#             # API 호출
#             response = model.generate_content(final_prompt)

#             # 안전 필터링 확인
#             if response.prompt_feedback.block_reason:
#                 print(f"  (ID: {article_id}) ⚠️ 'prompt'가 안전 필터에 차단됨 (이유: {response.prompt_feedback.block_reason})")
#                 return None # (재시도 안 함)
#             if not response.candidates:
#                 print(f"  (ID: {article_id}) ⚠️ 모델이 'candidates'를 생성 못함. (안전 문제)")
#                 return None # (재시도 안 함)

#             # JSON 파싱
#             result_data = json.loads(response.text)

#             # 성공
#             # 첫 시도가 아닌 재시도에서 성공한 경우 로깅
#             if attempt > 0:
#                 print(f"  (ID: {article_id}) ✅ {attempt + 1}차 시도 만에 성공.")

#             return result_data

#         except (ConnectionResetError, RemoteDisconnected) as net_error:
#             print(f"  (ID: {article_id}) ❗️ 네트워크 오류 감지 (시도 {attempt + 1}/{MAX_RETRIES}): {net_error}")

#             # 마지막 시도마저 실패하면, 포기하고 None 반환
#             if attempt + 1 == MAX_RETRIES:
#                 print(f"  (ID: {article_id}) ❌ 최종 재시도 실패. 이 기사를 건너뜁니다.")
#                 return None

#             # 대기
#             wait_time = BASE_DELAY_SECONDS * (2 ** attempt)
#             print(f"     ... {wait_time}초 후 재시도합니다 ...")
#             time.sleep(wait_time)
#             # (for 루프의 다음 시도로 넘어감)

#         #  JSON 파싱 오류
#         except json.JSONDecodeError as json_err:
#             print(f"  (ID: {article_id}) ❗️ JSON 파싱 오류 (모델이 스키마 준수 실패): {json_err}")
#             print(f"     (모델 원본 응답: {response.text[:150]}...)")
#             return None # (재시도 안 함)

#         # 그 외 모든 오류
#         except Exception as e:
#             error_message = str(e)
#             if "429" in error_message or "Resource Exhausted" in error_message:
#                  print(f"  (ID: {article_id}) ❌ API 할당량 초과(429) 감지. (time.sleep()을 늘려야 함)")
#             else:
#                 print(f"  (ID: {article_id}) ❌ API 호출 중 알 수 없는 오류: {e}")

#             return None # (재시도 안 함)

# print("✅ 'analyze_article_json_final' (자동 재시도) 함수 정의 완료.")

In [50]:
# 서버 오류(500) 재시도 포함 함수
print("\n--- 자동 재시도 함수 정의 ---")
print("   (네트워크 오류 + 서버 500/503/429 오류 시 3회 자동 재시도)")

# 네트워크 오류 처리를 위한 모듈 임포트
from http.client import RemoteDisconnected
# ConnectionResetError는 내장 오류

def analyze_article_json_final(article_id, article_title, article_content, model):
    """
    (v3 - 최종 완성) Gemini 모델을 호출하여 기사를 분석합니다.
    - [재시도 대상]
      1. 네트워크 오류 (ConnectionResetError, RemoteDisconnected)
      2. 서버 오류 (500, 503 Internal Error)
      3. 할당량 오류 (429 Resource Exhausted)
    - 위 오류 발생 시 '지수 백오프'를 적용하여 최대 3회 자동 재시도합니다.
    - [재시도 안 함]
      1. JSON 파싱 오류 (모델이 스키마 준수 실패)
      2. 안전 필터링
      3. 400 Bad Request (프롬프트 문제 등)
    """

    # 입력 값 검증
    if not all([article_title, article_content]):
        print(f"  (ID: {article_id}) ⚠️ 입력 값(제목 또는 본문)이 비어있어 건너뜁니다.")
        return None

    # 프롬프트 생성
    try:
        final_prompt = TEST_PROMPT_KR.replace("{title}", article_title) \
                                   .replace("{body}", article_content)
    except Exception as e:
        print(f"  (ID: {article_id}) ❌ 프롬프트 생성 오류: {e}")
        return None

    # 자동 재시도 루프 설정
    MAX_RETRIES = 3
    BASE_DELAY_SECONDS = 5 # (서버 오류는 더 긴 5초부터 시작)

    for attempt in range(MAX_RETRIES):
        try:
            # API 호출
            response = model.generate_content(final_prompt)

            # 안전 필터링 확인
            if response.prompt_feedback.block_reason:
                print(f"  (ID: {article_id}) ⚠️ 'prompt'가 안전 필터에 차단됨 (이유: {response.prompt_feedback.block_reason})")
                return None # (재시도 안 함)
            if not response.candidates:
                print(f"  (ID: {article_id}) ⚠️ 모델이 'candidates'를 생성 못함. (안전 문제)")
                return None # (재시도 안 함)

            # JSON 파싱
            result_data = json.loads(response.text)

            # 성공
            if attempt > 0:
                print(f"  (ID: {article_id}) ✅ {attempt + 1}차 시도 만에 성공.")

            return result_data

        # 네트워크 오류 (재시도)
        except (ConnectionResetError, RemoteDisconnected) as net_error:
            print(f"  (ID: {article_id}) ❗️ 네트워크 오류 감지 (시도 {attempt + 1}/{MAX_RETRIES}): {net_error}")
            # 아래 '재시도 로직'으로 이동

        # JSON 파싱 오류 (재시도 안 함)
        except json.JSONDecodeError as json_err:
            print(f"  (ID: {article_id}) ❗️ JSON 파싱 오류 (모델이 스키마 준수 실패): {json_err}")
            # response.text가 있어야만 이 오류가 나므로, 원본 응답 로깅 가능
            try:
                print(f"     (모델 원본 응답: {response.text[:150]}...)")
            except:
                pass # response 객체에 text가 없는 예외적인 경우
            return None # (루프 종료)

        # 그 외 모든 오류 (재시도 여부 결정)
        except Exception as e:
            error_message = str(e).lower()

            # 500(서버), 503(서비스), 429(할당량) 오류는 재시도
            if "500" in error_message or "internal error" in error_message or \
               "503" in error_message or "service unavailable" in error_message or \
               "429" in error_message or "resource exhausted" in error_message:

                print(f"  (ID: {article_id}) ❗️ 서버/할당량 오류(50x/429) 감지 (시도 {attempt + 1}/{MAX_RETRIES}): {error_message[:100]}...")
                # 아래 '재시도 로직'으로 이동

            else:
                # 400 Bad Request, 404 Not Found, 인증 오류 등은 재시도 안 함
                print(f"  (ID: {article_id}) ❌ 알 수 없는 오류 (재시도 불가): {e}")
                return None # (루프 종료)

        # 재시도 로직
        if attempt + 1 == MAX_RETRIES:
            print(f"  (ID: {article_id}) ❌ 최종 재시도 실패. 이 기사를 건너뜁니다.")
            return None # (루프 종료)

        # 5초, 10초, 20초... 대기
        wait_time = BASE_DELAY_SECONDS * (2 ** attempt)
        print(f"     ... {wait_time}초 후 재시도합니다 ...")
        time.sleep(wait_time)

print("✅ 'analyze_article_json_final' (v3 - 서버 오류 재시도) 함수 정의 완료.")


--- 자동 재시도 함수 정의 ---
   (네트워크 오류 + 서버 500/503/429 오류 시 3회 자동 재시도)
✅ 'analyze_article_json_final' (v3 - 서버 오류 재시도) 함수 정의 완료.


In [51]:
# # 기사 3개 확인용 코드

# %%time
# print(f"\n--- 3개 샘플 기사 분석 시작... ---")
# print(f"   (원본 {len(df)}개 중 3개만 가져와서 테스트합니다)")

# df_test = df.head(3)

# results_list_test = [] # 테스트 결과를 저장할 리스트
# original_ids_test = [] # 원본 article_id를 저장할 리스트

# # 'df' 대신 'df_test'로 루프를 실행
# for index, row in df_test.iterrows():
#     article_id = row['article_id']
#     title = row['title']
#     content = row['content']

#     # 함수 내부에서 이미 상세한 로그 출력
#     print(f"  ({index + 1}/{len(df_test)}) ID: {article_id} '{title[:30]}...' 분석 중...")

#     # 최종 재시도 함수 호출
#     analysis_result = analyze_article_json_final(article_id, title, content, model_json)

#     if analysis_result:
#         results_list_test.append(analysis_result)
#         original_ids_test.append(article_id)
#     else:
#         pass

#     # API 속도 제한
#     time.sleep(1.2)

# print(f"\n✅ 테스트 완료. {len(results_list_test)} / {len(df_test)}개 성공.")

# # 3개 샘플 결과 (수동 검토용)
# if not results_list_test:
#     print("❌ 분석된 테스트 결과가 없습니다. 함수 로그를 확인하세요.")
# else:
#     df_results_test = pd.DataFrame(results_list_test)
#     df_results_test['source_article_id'] = original_ids_test

#     print("\n--- 3개 샘플 분석 결과 (수동 검토용) ---")

#     from google.colab import data_table
#     data_table.enable_dataframe_formatter()

#     print(" (결과를 확인하세요. 'body' 열은 제외하고 표시됩니다)")
#     cols_to_show = [
#         'source_article_id', 'company', 'sentiment',
#         'sentiment_score', 'sentiment_reason', 'title'
#     ]
#     existing_cols = [col for col in cols_to_show if col in df_results_test.columns]

#     display(df_results_test[existing_cols])

In [52]:
# # 기사 10개 확인용 코드

# %%time
# # 10개 샘플 분석 (DB 저장 제외)
# print(f"\n--- 10개 샘플 분석 시작... ---")

# df_test_10 = df.head(10)

# results_list_test_10 = [] # 테스트 결과를 저장할 리스트
# original_ids_test_10 = [] # 원본 article_id를 저장할 리스트

# # 'df' 대신 'df_test_10'로 루프를 실행
# for index, row in df_test_10.iterrows():
#     article_id = row['article_id']
#     title = row['title']
#     content = row['content']

#     # 로그를 간소화 (ID만 출력)
#     print(f"  ({index + 1}/{len(df_test_10)}) ID: {article_id} 분석 중...", end='\r') # (end='\r'로 한 줄에 덮어쓰기)

#     # 최종 재시도 함수 호출
#     analysis_result = analyze_article_json_final(article_id, title, content, model_json)

#     if analysis_result:
#         results_list_test_10.append(analysis_result)
#         original_ids_test_10.append(article_id)
#     else:
#         # 함수 내부에서 이미 실패 원인을 출력함
#         pass

#     # API 속도 제한
#     time.sleep(1.2)

# print(f"\n✅ 100개 테스트 완료. {len(results_list_test_10)} / {len(df_test_10)}개 성공.")

# # 100개 샘플 결과 (수동 검토용)
# if not results_list_test_10:
#     print("❌ 분석된 테스트 결과가 없습니다. 함수 로그를 확인하세요.")
# else:
#     df_results_test_10 = pd.DataFrame(results_list_test_10)
#     df_results_test_10['source_article_id'] = original_ids_test_10

#     print("\n---10개 샘플 분석 결과 (수동 검토용) ---")

#     from google.colab import data_table
#     data_table.enable_dataframe_formatter()

#     print(" (10개의 결과만 샘플로 표시합니다. 전체 10개는 df_results_test_10 변수에 저장되어 있습니다.)")
#     cols_to_show = [
#         'source_article_id', 'company', 'sentiment',
#         'sentiment_score', 'sentiment_reason', 'title'
#     ]
#     existing_cols = [col for col in cols_to_show if col in df_results_test_10.columns]

#     # 10개를 다 출력하면 너무 기니, 상위 10개만 확인
#     display(df_results_test_10[existing_cols].head(10))


In [53]:
# # 기사 100개 확인용 코드

# %%time
# # 100개 샘플 분석 (DB 저장 제외)
# print(f"\n---  100개 샘플 분석 시작... ---")

# df_test_100 = df.head(100)

# results_list_test_100 = [] # 테스트 결과를 저장할 리스트
# original_ids_test_100 = [] # 원본 article_id를 저장할 리스트

# # 'df' 대신 'df_test_100'로 루프를 실행
# for index, row in df_test_100.iterrows():
#     article_id = row['article_id']
#     title = row['title']
#     content = row['content']

#     # 로그를 간소화 (ID만 출력)
#     print(f"  ({index + 1}/{len(df_test_100)}) ID: {article_id} 분석 중...", end='\r') # (end='\r'로 한 줄에 덮어쓰기)

#     # 최종 재시도 함수 호출
#     analysis_result = analyze_article_json_final(article_id, title, content, model_json)

#     if analysis_result:
#         results_list_test_100.append(analysis_result)
#         original_ids_test_100.append(article_id)
#     else:
#         # 함수 내부에서 이미 실패 원인을 출력함
#         pass

#     # API 속도 제한
#     time.sleep(1.2)

# print(f"\n✅ 100개 테스트 완료. {len(results_list_test_100)} / {len(df_test_100)}개 성공.")

# # 100개 샘플 결과 (수동 검토용)
# if not results_list_test_100:
#     print("❌ 분석된 테스트 결과가 없습니다. 함수 로그를 확인하세요.")
# else:
#     df_results_test_100 = pd.DataFrame(results_list_test_100)
#     df_results_test_100['source_article_id'] = original_ids_test_100

#     print("\n--- 5-C. 💎 100개 샘플 분석 결과 (수동 검토용) ---")

#     from google.colab import data_table
#     data_table.enable_dataframe_formatter()

#     print(" (상위 10개의 결과만 샘플로 표시합니다. 전체 100개는 df_results_test_100 변수에 저장되어 있습니다.)")
#     cols_to_show = [
#         'source_article_id', 'company', 'sentiment',
#         'sentiment_score', 'sentiment_reason', 'title'
#     ]
#     existing_cols = [col for col in cols_to_show if col in df_results_test_100.columns]

#     # 상위 10개만 확인
#     display(df_results_test_100[existing_cols].head(10))

#     # (팁) 전체 100개를 다 보고 싶다면 새 셀에서 'display(df_results_test_100)' 실행

In [54]:
# # JSON 파일로 다운로드

# import pandas as pd
# from google.colab import files
# import json

# # 파일 이름 정의
# filename = "analysis_results_100.json"

# # DataFrame이 존재하는지 확인
# try:
#     # 이 변수 이름은 테스트 코드의 변수명과 일치해야 함
#     if 'df_results_test_100' not in globals():
#         raise NameError("df_results_test_100 변수를 찾을 수 없습니다.")

#     print(f"'{filename}'으로 저장 준비 중... (DataFrame 크기: {len(df_results_test_100)} 건)")

#     # DataFrame을 JSON 문자열로 변환
#     json_string = df_results_test_100.to_json(
#         orient='records',
#         force_ascii=False,
#         indent=4
#     )

#     # Colab 환경에 파일로 저장
#     with open(filename, 'w', encoding='utf-8') as f:
#         f.write(json_string)

#     # Colab의 'files' 모듈을 사용해 브라우저로 다운로드
#     print(f"\n✅ {filename} 생성 완료.")
#     print("브라우저에서 다운로드 창이 뜰 때까지 잠시 기다려주세요...")
#     files.download(filename)

# except NameError as e:
#     print(f"❌ 오류: {e}")
# except Exception as e:
#     print(f"❌ 알 수 없는 오류 발생: {e}")

In [55]:
import pandas as pd
from sqlalchemy import text

print("--- [ 'analysisstatus' ENUM의 허용된 값 목록 조회 ] ---")

# PostgreSQL의 'pg_type'과 'pg_enum' 시스템 카탈로그를 조회하여 'analysisstatus'라는 ENUM에 정의된 모든 레이블(값)을 가져옵니다.
ENUM_QUERY = text("""
    SELECT e.enumlabel
    FROM pg_type t
    JOIN pg_enum e ON t.oid = e.enumtypid
    WHERE t.typname = 'analysisstatus'
    ORDER BY e.enumsortorder;
""")

try:
    if 'engine' not in globals():
        raise NameError("DB 'engine' 변수가 없습니다. 'DB 엔진 리셋' 셀을 다시 실행하세요.")

    df_enum_values = pd.read_sql(ENUM_QUERY, engine)

    if len(df_enum_values) == 0:
        print("❌ 'analysisstatus'라는 ENUM 타입을 찾을 수 없습니다. (이름을 다시 확인하세요)")
    else:
        print("✅✅✅ DB가 허용하는 'AnalysisStatus' 값 목록입니다. ✅✅✅")
        display(df_enum_values)

except Exception as e:
    print(f"❌ ENUM 값 조회 실패: {e}")
    print("   (DB 엔진이 'rollback' 오류 상태일 수 있습니다. 'DB 엔진 리셋' 셀을 다시 실행해 보세요)")

--- [ 'analysisstatus' ENUM의 허용된 값 목록 조회 ] ---
✅✅✅ DB가 허용하는 'AnalysisStatus' 값 목록입니다. ✅✅✅


,enumlabel
0,PENDING
1,RUNNING
2,COMPLETED
3,FAILED


In [56]:
# DB 엔진 재시작
import pandas as pd
from sqlalchemy import create_engine, text

# '고장 난' DB 엔진(연결 풀) 폐기
try:
    if 'engine' in globals():
        engine.dispose()
        print("✅ 기존의 '고장 난' DB 엔진(연결 풀)을 폐기했습니다.")
    else:
        print("⚠️ 'engine' 변수가 없지만, 새로 생성합니다.")

except Exception as e:
    print(f"❗️ 엔진 폐기 중 오류 발생 (무시하고 계속): {e}")

# 새 DB 엔진 생성
try:
    # (DATABASE_URL 변수는 이전에 이미 정의되어 있어야 함)
    engine = create_engine(DATABASE_URL)

    # 새 엔진으로 연결 테스트
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print(f"✅✅✅ 새 DB 엔진 연결 테스트 성공! (결과: {result.scalar()})")
        print("이제 DB 저장을 다시 시도할 수 있습니다.")

except Exception as e:
    print(f"❌ 새 DB 엔진 생성 실패: {e}")
    print("👉 DB 접속 정보나 Auth Proxy가 켜져 있는지 확인하세요.")

✅ 기존의 '고장 난' DB 엔진(연결 풀)을 폐기했습니다.
✅✅✅ 새 DB 엔진 연결 테스트 성공! (결과: 1)
이제 DB 저장을 다시 시도할 수 있습니다.


In [57]:
# SQL - DB에 저장되었는지 확인

print("\n--- [ article_analysis 테이블 상위 10개 확인 ] ---")
try:
    df_analy = pd.read_sql(text("SELECT * FROM article_analysis LIMIT 40"), engine)
    display(df_analy)
except Exception as e:
    print(f"❌ article_analysis 테이블 확인 실패: {e}")


--- [ article_analysis 테이블 상위 10개 확인 ] ---


,article_id,sentiment,sentiment_reason,sentiment_status,summary,summary_status,topic_status,last_processed_at
0,1,긍정 (Positive),"한·미 관세 협상 극적 타결로 자동차 관세율 인하, 대미 투자 약정 불확실성 해소,...",COMPLETED,None,PENDING,PENDING,None
1,2,긍정 (Positive),한미 관세 협상 타결로 자동차 관세가 25%에서 15%로 인하되어 현대차그룹의 연간...,COMPLETED,None,PENDING,PENDING,None
2,3,긍정 (Positive),"한미 관세협상 타결로 국내 수출 기업, 특히 자동차 업계의 고관세 부담이 해소되어 ...",COMPLETED,None,PENDING,PENDING,None
3,4,긍정 (Positive),대미 자동차 수출 관세가 25%에서 15%로 인하되면서 현대차그룹의 관세 비용 부담...,COMPLETED,None,PENDING,PENDING,None
4,5,긍정 (Positive),한미 관세 협상 타결로 미국 대 한국 자동차 품목 관세가 15%로 인하되어 현대차와...,COMPLETED,None,PENDING,PENDING,None
5,6,긍정 (Positive),한미 양국이 자동차 관세를 25%에서 15%로 낮추기로 합의하여 현대차그룹의 연간 ...,COMPLETED,None,PENDING,PENDING,None
6,7,긍정 (Positive),한미 관세 협상 타결로 미국 자동차 품목 관세가 15% 인하되면서 현대차와 기아의 ...,COMPLETED,None,PENDING,PENDING,None
7,8,긍정 (Positive),해외 IB가 코스피 6000선 도달을 전망하며 한국 증시의 상승 여력이 크다고 분석...,COMPLETED,None,PENDING,PENDING,None
8,9,긍정 (Positive),한미 관세 협상 타결로 현대차·기아 등 자동차 업계는 연간 3조원 이상 비용 절감 ...,COMPLETED,None,PENDING,PENDING,None
9,10,긍정 (Positive),한-미 관세 협상 타결로 자동차·부품 관세가 인하되어 주가가 14% 상승하는 등 긍...,COMPLETED,None,PENDING,PENDING,None


In [58]:
import pandas as pd
from sqlalchemy import text, create_engine
import sys
import time

try:
    print("... DB 엔진 리셋 시도 ...")
    # 1. Auth Proxy 재실행 (Connection refused 대비)
    get_ipython().system_raw(
        f'./cloud_sql_proxy -instances={INSTANCE_CONNECTION_NAME}=tcp:{DB_PORT} &'
    )
    time.sleep(5)

    # 2. 엔진 폐기 및 재생성
    engine.dispose()
    engine = create_engine(DATABASE_URL)

    print("✅ 엔진 리셋 및 Auth Proxy 재시작 완료.")

except Exception as e:
    print(f"❌ 엔진 리셋 실패: {e}")
    sys.exit()

# (DB 총 개수 확인)
try:
    with engine.connect() as conn:
        count_result = conn.execute(text("SELECT COUNT(*) FROM article_analysis"))
        total_rows = count_result.scalar()
        print(f"\n✅✅✅ 현재 'article_analysis' 테이블에 저장된 총 분석 건수: {total_rows} 건")

except Exception as e:
    print(f"❌ DB 확인 실패: {e}")

... DB 엔진 리셋 시도 ...
✅ 엔진 리셋 및 Auth Proxy 재시작 완료.

✅✅✅ 현재 'article_analysis' 테이블에 저장된 총 분석 건수: 75 건


In [59]:
# --- 'v3' 재시도 함수 정의 ---
print("\n--- 'v3' 재시도 함수 정의 ---")
print("   (네트워크 + 서버 50x/429 오류 시 3회 자동 재시도)")

# 필요한 모듈 임포트
from http.client import RemoteDisconnected
import time
import sys
import json
import pandas as pd
from sqlalchemy import types, create_engine, text
import numpy as np

def analyze_article_json_final(article_id, article_title, article_content, model):
    """
    Gemini 모델을 호출하여 기사를 분석합니다.
    - [재시도 대상]
      1. 네트워크 오류 (ConnectionResetError, RemoteDisconnected)
      2. 서버 오류 (500, 503 Internal Error)
      3. 할당량 오류 (429 Resource Exhausted)
    - 위 오류 발생 시 '지수 백오프'를 적용하여 최대 3회 자동 재시도합니다.
    - [재시도 안 함]
      1. JSON 파싱 오류 (모델이 스키마 준수 실패)
      2. 안전 필터링
      3. 400 Bad Request (프롬프트 문제 등)
    """

    # --- 입력 값 검증 ---
    if not all([article_title, article_content]):
        print(f"  (ID: {article_id}) ⚠️ 입력 값(제목 또는 본문)이 비어있어 건너뜁니다.")
        return None

    # --- 프롬프트 생성 ---
    try:
        final_prompt = TEST_PROMPT_KR.replace("{title}", article_title) \
                                   .replace("{body}", article_content)
    except Exception as e:
        print(f"  (ID: {article_id}) ❌ 프롬프트 생성 오류: {e}")
        return None

    # --- 자동 재시도 루프 설정 ---
    MAX_RETRIES = 3
    BASE_DELAY_SECONDS = 10 # (★ 10초, 20초, 40초로 재시도 간격 늘림 ★)

    for attempt in range(MAX_RETRIES):
        try:
            # --- API 호출 ---
            response = model.generate_content(final_prompt)

            # --- 안전 필터링 확인 ---
            if response.prompt_feedback.block_reason:
                print(f"  (ID: {article_id}) ⚠️ 'prompt'가 안전 필터에 차단됨 (이유: {response.prompt_feedback.block_reason})")
                return None # (재시도 안 함)
            if not response.candidates:
                print(f"  (ID: {article_id}) ⚠️ 모델이 'candidates'를 생성 못함. (안전 문제)")
                return None # (재시도 안 함)

            # --- JSON 파싱 ---
            result_data = json.loads(response.text)

            # 성공
            if attempt > 0:
                print(f"  (ID: {article_id}) ✅ {attempt + 1}차 시도 만에 성공.")

            return result_data

        # 네트워크 오류
        except (ConnectionResetError, RemoteDisconnected) as net_error:
            print(f"  (ID: {article_id}) ❗️ 네트워크 오류 감지 (시도 {attempt + 1}/{MAX_RETRIES}): {net_error}")

        # JSON 파싱 오류
        except json.JSONDecodeError as json_err:
            print(f"  (ID: {article_id}) ❗️ JSON 파싱 오류 (모델이 스키마 준수 실패): {json_err}")
            try:
                print(f"     (모델 원본 응답: {response.text[:150]}...)")
            except:
                pass
            return None # 루프 종료

        # 그 외 모든 오류 (재시도 여부 결정)
        except Exception as e:
            error_message = str(e).lower()

            if "500" in error_message or "internal error" in error_message or \
               "503" in error_message or "service unavailable" in error_message or \
               "429" in error_message or "resource exhausted" in error_message:

                print(f"  (ID: {article_id}) ❗️ 서버/할당량 오류(50x/429) 감지 (시도 {attempt + 1}/{MAX_RETRIES}): {error_message[:100]}...")

            else:
                print(f"  (ID: {article_id}) ❌ 알 수 없는 오류 (재시도 불가): {e}")
                return None # 루프 종료

        # 재시도 로직
        if attempt + 1 == MAX_RETRIES:
            print(f"  (ID: {article_id}) ❌ 최종 재시도 실패. 이 기사를 건너뜁니다.")
            return None # 루프 종료

        wait_time = BASE_DELAY_SECONDS * (2 ** attempt)
        print(f"     ... {wait_time}초 후 재시도합니다 ...")
        time.sleep(wait_time)

print("✅ 'analyze_article_json_final' (v3 - 10초 재시도) 함수 정의 완료.")


--- 'v3' 재시도 함수 정의 ---
   (네트워크 + 서버 50x/429 오류 시 3회 자동 재시도)
✅ 'analyze_article_json_final' (v3 - 10초 재시도) 함수 정의 완료.


In [60]:
# 작업 중단 시 '재시작'용 확인 코드
print("작업 중단 시 '재시작'용 확인 코드")
print("(Auth Proxy, DB 엔진 리셋, 총 개수 확인)")

import pandas as pd
from sqlalchemy import text, create_engine
import sys
import time

try:
    print("DB 엔진 리셋 시도")
    get_ipython().system_raw(
        f'./cloud_sql_proxy -instances={INSTANCE_CONNECTION_NAME}=tcp:{DB_PORT} &'
    )
    time.sleep(5)
    print("Auth Proxy 재실행 시도 완료")

    engine.dispose()
    engine = create_engine(DATABASE_URL)

    print("✅ 엔진 리셋 및 Auth Proxy 재시작 완료.")
except Exception as e:
    print(f"❌ 엔진 리셋 실패: {e}")
    sys.exit()

try:
    with engine.connect() as conn:
        count_result = conn.execute(text("SELECT COUNT(*) FROM article_analysis"))
        total_rows = count_result.scalar()
        print(f"\n✅✅✅ 현재 'article_analysis' 테이블에 저장된 총 분석 건수: {total_rows} 건")
        print(f"'START_INDEX'를 이 숫자({total_rows})로 수정하세요.")
except Exception as e:
    print(f"❌ DB 확인 실패: {e}")

작업 중단 시 '재시작'용 확인 코드
(Auth Proxy, DB 엔진 리셋, 총 개수 확인)
DB 엔진 리셋 시도
Auth Proxy 재실행 시도 완료
✅ 엔진 리셋 및 Auth Proxy 재시작 완료.

✅✅✅ 현재 'article_analysis' 테이블에 저장된 총 분석 건수: 75 건
'START_INDEX'를 이 숫자(75)로 수정하세요.


In [61]:
%%time

# 중단된 지점부터 시작
# 위 셀에서 확인한 '총 분석 건수'로 이 숫자를 바꾸기
START_INDEX = 75

try:
    df_remaining = df.iloc[START_INDEX:]
except (IndexError, NameError):
    print(f"❌ 'df' DataFrame을 찾을 수 없습니다. 'DB에서 기사 데이터 로드' 셀을 다시 실행하세요.")
    sys.exit()

if len(df_remaining) == 0:
    print("✅ 모든 기사가 이미 처리되었습니다. (남은 기사 0개)")
    sys.exit()

print(f"\n--- (★최종 실행★) 남은 {len(df_remaining)}개 기사 전체 분석 시작 ---")


# 배치 크기 10개
BATCH_SIZE = 10
print(f"   (안전 모드: {BATCH_SIZE}개씩 자주 저장합니다.)")

#  API 속도 5.0초
API_CALL_DELAY = 5.0
print(f"   (API 할당량(429/503) 방지를 위해 {API_CALL_DELAY}초마다 호출합니다.)")


# DB 설정은 최종본
DB_COLUMNS = [
    'article_id', 'sentiment', 'sentiment_reason',
    'sentiment_status', 'summary_status', 'topic_status'
]
DTYPE_MAPPING_DB = {
    'article_id': types.BIGINT, 'sentiment': types.VARCHAR(100),
    'sentiment_reason': types.TEXT, 'sentiment_status': types.VARCHAR(50),
    'summary_status': types.VARCHAR(50), 'topic_status': types.VARCHAR(50)
}
ENUM_COMPLETED_VALUE = 'COMPLETED'
ENUM_PENDING_VALUE = 'PENDING'

# 배치 설정 및 메인 루프
try:
    total_batches = int(np.ceil(len(df_remaining) / BATCH_SIZE))
    print(f"\n총 {len(df_remaining)}개의 기사를 {BATCH_SIZE}개씩, {total_batches}개의 배치로 나눠 처리합니다.")
except Exception as e:
    print(f"❌ 'df_remaining' DataFrame 분할 실패: {e}")
    raise e

all_batches_df = np.array_split(df_remaining, total_batches)
total_success_count = 0

for i, batch_df in enumerate(all_batches_df):

    sys.stdout.flush()
    print(f"\n--- 🚀 배치 {i + 1}/{total_batches} 시작 ({len(batch_df)}개) ---")

    results_list_batch = []
    original_ids_batch = []

    # (내부) 기사 분석 루프 (API 호출)
    for index, row in batch_df.iterrows():
        article_id = row['article_id']
        title = row['title']
        content = row['content']

        print(f"  (배치 {i+1}) ID: {article_id} 분석 중...", end='\r')

        analysis_result = analyze_article_json_final(article_id, title, content, model_json)

        if analysis_result:
            results_list_batch.append(analysis_result)
            original_ids_batch.append(article_id) # 원본 ID도 따로 저장
        else:
            pass

        # 429 방지를 위한 API 속도 제한
        time.sleep(API_CALL_DELAY)

    print(f"\n✅ 배치 {i + 1} 분석 완료. {len(results_list_batch)} / {len(batch_df)}개 성공.")
    total_success_count += len(results_list_batch)

    # 10개 배치 결과를 DB에 즉시 저장
    if not results_list_batch:
        print("  ⚠️ 이번 배치에서 분석 성공한 데이터가 없습니다. DB 저장을 건너뜁니다.")
        continue

    df_results_batch = pd.DataFrame(results_list_batch)
    df_results_batch['article_id'] = original_ids_batch

    df_results_batch['sentiment_status'] = ENUM_COMPLETED_VALUE
    df_results_batch['summary_status'] = ENUM_PENDING_VALUE
    df_results_batch['topic_status'] = ENUM_PENDING_VALUE

    try:
        df_final_to_save = df_results_batch[DB_COLUMNS]
    except KeyError as e:
        print(f"❌ DB 저장 실패 (KeyError): {e} ...")
        print(f"   (현재 생성된 컬럼: {df_results_batch.columns.to_list()})")
        continue

    # DB 저장
    try:
        print(f"  ... DB 'article_analysis' 테이블에 {len(df_final_to_save)}개 저장 시도 ...")

        df_final_to_save.to_sql(
            'article_analysis', con=engine,
            if_exists='append', index=False, dtype=DTYPE_MAPPING_DB
        )
        print(f"✅✅✅ 배치 {i + 1} 결과가 DB에 성공적으로 저장되었습니다. ✅✅✅")

    # 자가 치유
    except Exception as e:
        print(f"❌❌❌ 배치 {i + 1} DB 저장 실패: {e}")

        error_msg = str(e).lower()
        if "rollback" in error_msg or "connection refused" in error_msg:
            print("❗️ 'rollback' 또는 'Connection refused' 오류 감지. DB 엔진을 리셋합니다...")
            try:
                get_ipython().system_raw(
                    f'./cloud_sql_proxy -instances={INSTANCE_CONNECTION_NAME}=tcp:{DB_PORT} &'
                )
                time.sleep(5)
                engine.dispose()
                engine = create_engine(DATABASE_URL)
                with engine.connect() as conn:
                    conn.execute(text("SELECT 1"))
                print("   ✅✅✅ 새 엔진 생성 및 연결 테스트 성공! 다음 배치를 계속합니다.")
            except Exception as e2:
                print(f"   ❌❌❌ 엔진 '자가 치유' 실패: {e2}")
        else:
             print("   (이번 배치의 데이터가 손실되었습니다. 다음 배치를 계속합니다.)")

# 최종 완료
print(f"\n\n🎉🎉🎉 모든 {total_batches}개 배치 처리가 완료되었습니다! 🎉🎉🎉")


--- (★최종 실행★) 남은 2619개 기사 전체 분석 시작 ---
   (안전 모드: 10개씩 자주 저장합니다.)
   (API 할당량(429/503) 방지를 위해 5.0초마다 호출합니다.)

총 2619개의 기사를 10개씩, 262개의 배치로 나눠 처리합니다.

--- 🚀 배치 1/262 시작 (10개) ---


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2306.41ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 19661.39ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2535.49ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 7679.56ms


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 15030.29ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 3669.62ms


KeyboardInterrupt: 